In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch

from gluonts.dataset.repository.datasets import get_dataset
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import make_evaluation_predictions, Evaluator

# from gluonts.torch.model.lag_tst.estimator import LagTSTEstimator
from gluonts.torch.distributions import NegativeBinomialOutput
from gluonts.torch.modules.loss import NegativeLogLikelihood

from LagTST import LagTSTEstimator

# from pts.modules import StudentTOutput

import random
import numpy as np
import time
import optuna

In [2]:
class LagTSTObjective:
    def __init__(self, dataset, metric_type="mean_wQuantileLoss"):
        self.dataset = dataset
        self.metric_type = metric_type

    def get_params(self, trial) -> dict:
        return {
            "context_length": trial.suggest_int(
                "context_length",
                dataset.metadata.prediction_length,
                dataset.metadata.prediction_length * 10,
                4,
            ),
            "batch_size": trial.suggest_int("batch_size", 128, 256, 64),
            "d_model": trial.suggest_int("d_model", 16, 64, 16),
            "dim": trial.suggest_int("dim", 16, 64, 16),
            "patch_size": trial.suggest_int("dim", 2, 16, 4),
            "kernel_size": trial.suggest_int("dim", 9, 18, 3),
        }

    def __call__(self, trial):
        params = self.get_params(trial)

        estimator = LagTSTEstimator(
            prediction_length=self.dataset.metadata.prediction_length,
            context_length=dataset.metadata.prediction_length,
            freq=dataset.metadata.freq,
            scaling="std",
            # distr_output=NegativeBinomialOutput(),
            # loss=NegativeLogLikelihood(beta=0.2),
            d_model=params["d_model"],
            dim_feedforward=params["dim"],
            batch_size=params["batch_size"],
            num_batches_per_epoch=100,
            trainer_kwargs=dict(accelerator="gpu", max_epochs=30),
        )
        predictor = estimator.train(
            training_data=self.dataset.train,
            cache_data=True,
            shuffle_buffer_length=1024,
            validation_data=self.dataset.test,
        )

        forecast_it, ts_it = make_evaluation_predictions(
            dataset=dataset.test,
            predictor=predictor,
        )
        forecasts = list(forecast_it)
        # if layer == layers[0]:
        tss = list(ts_it)
        evaluator = Evaluator()
        agg_metrics, _ = evaluator(iter(tss), iter(forecasts))
        return agg_metrics[self.metric_type]

In [3]:
dataset = get_dataset(
    "solar-energy", regenerate=False
)  # dataset = get_dataset("electricity")

In [ ]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)
start_time = time.time()
study = optuna.create_study(direction="minimize")
study.optimize(LagTSTObjective(dataset), n_trials=10)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
print(time.time() - start_time)

[I 2023-04-28 02:51:34,243] A new study created in memory with name: no-name-c46306ac-2557-4512-97d2-c011e7964cba
/ccs/proj/csc499/hstellar/py38/lib/python3.8/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [2, 16] and step=4, but the range is not divisible by `step`. It will be replaced by [2, 14].
  warnings.warn(
/ccs/proj/csc499/hstellar/py38/lib/python3.8/site-packages/optuna/trial/_trial.py:681: RuntimeWarning: Inconsistent parameter values for distribution with name "dim"! This might be a configuration mistake. Optuna allows to call the same distribution with the same name more than once in a trial. When the parameter values are inconsistent optuna only uses the values of the first call and ignores all following. Using these values: {'log': False, 'step': 16, 'low': 16, 'high': 64}
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, usin

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 100: 'val_loss' reached 3.78605 (best 3.78605), saving model to '/gpfs/alpine/csc499/scratch/hstellar/ConvTs/ConvTS-Mixer-main/lightning_logs/version_14/checkpoints/epoch=0-step=100.ckpt' as top 1


...

Epoch 27, global step 2800: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 28, global step 2900: 'val_loss' was not in top 1


Validation: 0it [00:00, ?it/s]

Epoch 29, global step 3000: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=30` reached.
Running evaluation: 959it [00:01, 718.76it/s]
